# Imports

Import python libraries as well as the self written FERMI library.

In [ ]:
import sys, os
from os.path import join, split
from getpass import getuser
from glob import glob
from time import strftime
from importlib import reload
from copy import deepcopy
from tqdm.auto import tqdm

# data
import numpy as np
import xarray as xr
import pandas as pd
import h5py

# Images
import imageio
from imageio import imread

# Plotting
import matplotlib.pyplot as plt
from matplotlib.image import NonUniformImage
import matplotlib.gridspec as gridspec
from matplotlib.path import Path

# pyFAI
import pyFAI

pyFAI.disable_opencl = True  # get rid of annoying warning ;)
from pyFAI.azimuthalIntegrator import AzimuthalIntegrator
from pyFAI.detectors import Detector

# Scipy
from scipy.ndimage.filters import median_filter

# Self-written libraries
import process_FERMI as pf
from process_FERMI import interactive
from proces_FERMI.interactive import cimshow

In [ ]:
# interactive plotting
import ipywidgets

%matplotlib widget
plt.rcParams["figure.constrained_layout.use"] = True

# Auto formatting of cells
%load_ext jupyter_black

## Functions

In [ ]:
def create_folder(folder):
    """
    Creates input folder if it does not exist yet
    """

    if not (os.path.exists(folder)):
        print("Creating folder " + folder)
        os.makedirs(folder)
    return folder

In [ ]:
def preprocess_exp(datafolder, extension, keys=None, sort=False, full_rate=False):
    # Loading experiment data
    print("Loading: %s" % (datafolder + extension))
    exp = pf.get_exp_dataframe(datafolder + extension, keys=keys)
    for k in ["xgm_UH", "xgm_SH", "diode_sum"]:
        exp[k + "_sum"] = exp[k].apply(np.sum)

    exp["diode_sum_mean"] = exp.diode_sum.apply(np.mean)
    exp["diode_sum_sum"] = exp.diode_sum.apply(np.sum)
    exp["diode_sum_std"] = exp.diode_sum.apply(np.std)
    exp["IR_mean"] = exp.IR.apply(np.mean)
    exp["IR_std"] = exp.IR.apply(np.std)
    exp["magnet_mean"] = exp.magnet.apply(np.mean)
    exp["magnet_mean"] = exp.magnet_mean.apply(np.round, args=(3,))
    exp["bunchid"] = exp.bunches.apply(lambda l: l[-1])

    return exp

In [ ]:
# Draw circle mask
def circle_mask(shape, center, radius, sigma="none"):
    """
    Draws circle mask with option to apply gaussian filter for smoothing

    Parameter
    =========
    shape : int tuple
        shape/dimension of output array
    center : int tuple
        center coordinates (ycenter,xcenter)
    radius : scalar
        radius of mask in px. Care: diameter is always (2*radius+1) px
    sigma : scalar
        std of gaussian filter

    Output
    ======
    mask: array
        binary mask, or smoothed binary mask
    ======
    author: ck 2022
    """

    # setup array
    x = np.linspace(0, shape[1] - 1, shape[1])
    y = np.linspace(0, shape[0] - 1, shape[0])
    X, Y = np.meshgrid(x, y)

    # define circle
    mask = np.sqrt(((X - center[1]) ** 2 + (Y - center[0]) ** 2)) <= (radius)
    mask = mask.astype(float)

    # smooth aperture
    if sigma != "none":
        mask = scipy.ndimage.filters.gaussian_filter(mask, sigma)

    return mask


def create_single_polygon_mask(shape, coordinates):
    """
    Creates a polygon mask from coordinates of corner points

    Parameter
    =========
    shape : int tuple
        shape/dimension of output array
    coordinates: nested list
        coordinates of polygon corner points [[yc_1,xc_1],[yc_2,xc_2],...]


    Output
    ======
    mask: array
        binary mask where filled polygon is "1"
    ======
    author: ck 2023
    """

    x, y = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]))
    x, y = x.flatten(), y.flatten()

    points = np.vstack((x, y)).T

    path = Path(coordinates)
    mask = path.contains_points(points)
    mask = mask.reshape(shape)
    return mask


def create_polygon_mask(shape, coordinates):
    """
    Creates multiple polygon masks from set of coordinates of corner points

    Parameter
    =========
    shape : int tuple
        shape/dimension of output array
    coordinates: nested list
        coordinates of polygon corner points for multiple polygons
        [[[yc_1,xc_1],[yc_2,xc_2],...],[[yc_1,xc_1],[yc_2,xc_2],...]]

    Output
    ======
    mask: array
        binary mask where filled polygons are "1"
    ======
    author: ck 2023
    """

    if len(coordinates) == 1:
        mask = create_single_polygon_mask(shape, coordinates[0])
    elif len(coordinates) > 1:
        mask = np.zeros(shape)
        for coord in coordinates:
            mask = mask + create_single_polygon_mask(shape, coord)
            mask[mask > 1] = 1

    return mask


def load_poly_masks(polygon_name_list, shape):
    """
    Loads set of polygon masks based on stored coordinates

    Parameter
    =========
    polygon_name_list : list
        shape/dimension of output array


    Output
    ======
    mask: array
        binary mask where filled polygons are "1"
    ======
    author: ck 2023
    """

    mask = []
    for polygon_name in polygon_name_list:
        coord = load_poly_coordinates(polygon_name)
        mask.append(create_polygon_mask(shape, [coord]).astype(float))

    mask = np.array(mask)
    mask = np.sum(mask, axis=0)
    mask[mask > 1] = 1

    return mask

# Experimental details

In [ ]:
# Define basic folders
BASEFOLDER = r"/net/online4diproi/store/"
PROPOSAL = "20224053"
basefolder = join(BASEFOLDER, PROPOSAL)
USER = getuser()

In [ ]:
# Dict with most basic experimental parameter
experimental_setup = {
    "ccd_dist": 0.103,  # ccd to sample distance
    "px_size": 11e-6,  # pixel_size of camera
    "binning": 1,  # Camera binning
}

# Setup for azimuthal integrator
detector = Detector(
    experimental_setup["binning"] * experimental_setup["px_size"],
    experimental_setup["binning"] * experimental_setup["px_size"],
)

# General saving folder
folder_general = create_folder(join(basefolder, "results", "processed", USER))
print("Output Folder: %s" % folder_general)

# Load Data

## Define Scan ids for loading

In [ ]:
# Define for loading
sample = "S2205ref"
membrane = "I7"
scan_id = 185
IR_delay = 500
scan = f"%s_XPCS_IR%03dps_ScanB_%03d" % (membrane, IR_delay, scan_id)
scan_axis = "delay_ACDC"
full_rate = True

# Time-zero for delay scans
t0 = 9.925

# Folder for loading
samplefolder = join(sample, scan)
datafolder = join(basefolder, samplefolder)
extra_keys = {
    "diode_sum": "PAM/FQPDSum",
    "IR": "Laser/Energy1",
    "magnet": "DPI/CoilCurrent",
    "bunches": "bunches",
    "time": "",
    "global_delay": "Laser/DelayTotem",
}

# Create savefolder
fsave = create_folder(join(folder_general, sample, membrane))

## Pumped images

In [ ]:
def filter_images(data):
    # Filter empty images
    data["images_max"] = data.images.max(["y", "x"])
    data = data.where(data["images_max"] > 1)

    # Filter data with too high or low fel intensity
    data = data.where(
        np.abs(data[filter_key] - data[filter_key].mean("index"))
        < 2 * data[filter_key].std("index")
    )

    # Calc Averaged images and intensity
    data["images_mean"] = data["images"].mean("index")
    data[filter_key + "_sum"] = data[filter_key].sum("index")

    return data

In [ ]:
def process_images(exp):
    # Loading image data
    data = xr.Dataset()
    data["images"] = xr.DataArray(
        [
            pf.loadh5(fname, extra_keys=["alignz", "PAM/FQPDSum"], full_rate=full_rate)[
                0
            ].astype(np.uint16)
            for fname in exp["filename"]
        ],
        dims=["delay_idx", "index", "y", "x"],
    )
    data[scan_axis] = xr.DataArray(exp[scan_axis], dims=["delay_idx"])
    data[filter_key] = xr.DataArray(
        np.stack(exp[filter_key]), dims=["delay_idx", "index"]
    )

    # Filter images
    data = filter_images(data)

    data["time"] = xr.DataArray(exp["time"], dims=["delay_idx"])

    return data

In [ ]:
# Loading experiment data
end = 5
filter_key = "diode_sum"
norm_key = "diode_sum"
extension = ""
exp = preprocess_exp(datafolder, extension, keys=extra_keys, full_rate=full_rate)
exp = exp[:end]

# delay scan
exp["delay_ps"] = 6.67 * (exp["delay"] - t0) + (600 - exp["global_delay"])
exp = exp.sort_values(scan_axis)

# Add wavelength
experimental_setup["lambda"] = exp["wavelength"][0] * 1e-9

# Load data
data = process_images(exp)
data[scan_axis] = data[scan_axis].mean("index")

print("Data loaded!")

In [ ]:
# What did you scan?
fig, ax = plt.subplots()
ax.plot(np.arange(len(data[scan_axis])), data[scan_axis], "-o")
ax.set_xlabel("Index")
ax.set_ylabel(scan_axis)
ax.grid()

In [ ]:
# Plot images
fig, ax = cimshow(data["images_mean"])
fig.set_size_inches(6, 6)
ax.set_title("Pumped Images")

## Dark images

In [ ]:
# Loading experiment data
extension = "_BG"
exp_bg = preprocess_exp(datafolder, extension, keys=extra_keys)
exp_bg = exp_bg.sort_values("time")

# Loading image data
dark = process_images(exp_bg)

# Drop single frames
dark = dark.drop_dims("index")
dark = dark.swap_dims({"delay_idx": "time"})

print("Data loaded!")

In [ ]:
# Plot images
fig, ax = cimshow(dark["images_mean"])
fig.set_size_inches(6, 6)
ax.set_title("Dark Images")

## Laser only

In [ ]:
# Loading experiment data
extension = "_OL"
exp_ol = preprocess_exp(datafolder, extension, keys=extra_keys)
exp_ol = exp_ol.sort_values("time")

# Loading image data
dark_ol = process_images(exp_ol)

# Drop single frames
dark_ol = dark_ol.drop_dims("index")
dark_ol = dark_ol.swap_dims({"delay_idx": "time"})

print("Data loaded!")

In [ ]:
# Plot images
fig, ax = cimshow(dark_ol["images_mean"])
fig.set_size_inches(6, 6)
ax.set_title("Only Laser Images")

## FEL only

In [ ]:
# Loading experiment data
extension = "_OF"
exp_of = preprocess_exp(datafolder, extension, keys=extra_keys)
exp_of = exp_of.sort_values("time")

# Loading image data
dark_of = process_images(exp_of)

# Drop single frames
dark_of = dark_of.drop_dims("index")
dark_of = dark_of.swap_dims({"delay_idx": "time"})

print("Data loaded!")

In [ ]:
# Plot images
fig, ax = cimshow(dark_of["images_mean"])
fig.set_size_inches(6, 6)
ax.set_title("Only FEL Images")

# Preprocessing

## Normalize images

In [ ]:
# Subtract only laser background from pumped image
for i, delay in enumerate(tqdm(data[scan_axis].values)):
    data["images"][i] = data["images"][i] - (
        dark_ol.sel(time=data.time[i], method="nearest").images_mean
    )

    # Subtract dark background from only fel image
    dark_of["images_mean"][i] = (
        dark_of["images_mean"][i]
        - dark_of.sel(time=dark.time[i], method="nearest").images_mean
    )

# Normalize to laser fluences
data["images"] = data["images"] / data[norm_key]
dark_of["images_mean"] = dark_of["images_mean"] / dark_of[norm_key + "_sum"]

# Subtract final background
for i, delay in enumerate(tqdm(data[scan_axis].values)):
    data["images"][i] = data["images"][i] - (
        dark_of.sel(time=data.time[i], method="nearest").images_mean
    )
    
# Calc average image for each delay
data["images_mean"] = data["images"].mean("index")
data["im_mean"] = data.images_mean.mean("delay_idx")
im_mean = data["im_mean"].values

In [ ]:
# Plot images
fig, ax = cimshow(data.images[2])
fig.set_size_inches(6, 6)
ax.set_title("Normalized Images")

In [ ]:
# Plot images
fig, ax = cimshow(data.images_mean)
fig.set_size_inches(6, 6)
ax.set_title("Normalized Images")

In [ ]:
# Plot images
fig, ax = cimshow(data.images_mean.mean("delay_idx"))
fig.set_size_inches(6, 6)
ax.set_title("Normalized Images")

## Draw beamstop mask

In [ ]:
poly_mask = interactive.draw_polygon_mask(im_mean)

In [ ]:
# Take poly coordinates and mask from widget
p_coord = poly_mask.coordinates
mask = poly_mask.full_mask.astype(int)

cimshow(mask)

print("Mask coordinates: %s" % p_coord)

In [ ]:
def load_poly_coordinates(polyname):
    if polyname == "bs_cross":
        coord = [
            (946.9258289571189, 901.9078818510652),
            (-13.52166457167948, 915.628560330048),
            (-8.033393180086364, 1107.7180590358075),
            (944.1816932613224, 1099.4856519484179),
            (946.9258289571189, 1988.5856173865054),
            (945.4332858438797, 2042.780748713184),
            (1134.4167167610876, 2056.6765892218023),
            (1145.5333891679823, 1092.305257923697),
            (2007.0755007023129, 1106.2010984323151),
            (2067.2340362524137, 1100.0546685633194),
            (2067.2340362524137, 910.7093055533562),
            (1147.9485781605636, 902.4768984659665),
            (1153.4368495521567, -38.761645192255855),
            (955.8590794548038, -22.296831017476507),
        ]
    elif polyname == "bs_bar_delayscans":
        coord = [
            (2067.431746885335, 938.380826986908),
            (1998.8347372592004, 924.6614251397921),
            (1946.7010099433378, 938.3808270650193),
            (201.59308505446313, 954.8441093752917),
            (-34.0757676448203, 955.1489845092347),
            (-28.588006874729558, 1158.1961350015144),
            (945.4895298163876, 1147.2206134613327),
            (2062.858618967594, 1138.3792216748711),
        ]

    return coord

In [ ]:
# Which drawn masks do you want to load? (you can add multiple masks in list e.g. ["bs_cross","bs_bar_delayscans"])
polygon_names = ["bs_cross", "bs_bar_delayscans"]
mask = load_poly_masks(polygon_names, im_mean.shape)

fig, ax = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True)
mi, ma = np.percentile(im_mean, [1, 99])
ax[0].imshow(im_mean * (1 - mask), vmin=mi, vmax=ma)
ax[0].set_title("(1-mask)")
ax[1].imshow(im_mean * mask, vmin=mi, vmax=ma)
ax[1].set_title("mask")
plt.tight_layout()

## Find center

### Basic widget to find center

Try to **align** the circles to the **center of the scattering pattern**. Care! Position of beamstop might be misleading and not represent the actual center of the hologram. 

In [ ]:
# Set center position via widget
ic = interactive.InteractiveCenter(im_mean)

In [ ]:
# Get center positions
center = [ic.c0, ic.c1]
print(f"Center:", center)

### Azimuthal integrator widget for finetuning

In [ ]:
# Setup azimuthal integrator for virtual geometry
ai = interactive.AzimuthalIntegrator(
    dist=experimental_setup["ccd_dist"],
    detector=detector,
    wavelength=experimental_setup["lambda"],
    poni1=center[0]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # y (vertical)
    poni2=center[1]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # x (horizontal)
)

In [ ]:
# Plotting to find  relevant q range
I_t, q_t, phi_t = ai.integrate2d(
    im_mean,
    200,
    radial_range=(0, 0.07),
    unit="q_nm^-1",
    correctSolidAngle=False,
    dummy=np.nan,
    mask=mask,
)
az2d = xr.DataArray(I_t, dims=("phi", "q"), coords={"q": q_t, "phi": phi_t})

# Plot
fig, ax = plt.subplots()
mi, ma = np.nanpercentile(I_t, [1, 99])
az2d.plot.imshow(ax=ax, vmin=mi, vmax=ma)
plt.title(f"Azimuthal integration")

# Vertical lines
# q_lines = [0.025, 0.05]
# for qt in q_lines:
#    ax.axvline(qt, ymin=0, ymax=180, c="red")

In [ ]:
aic = interactive.AzimuthalIntegrationCenter(
    # np.log10(im_mean - np.min(im_mean) + 1),
    im_mean,
    ai,
    c0=center[0],
    c1=center[1],
    mask=mask,
    im_data_range=[1, 95],
    radial_range=(0.003, 0.07),
    qlines=[40, 60],
)

In [ ]:
# Get center positions
center = [aic.c0, aic.c1]
print(f"Center:", center)

# Azimuthal Integration

In [ ]:
# Setup final azimuthal integrator for virtual geometry
ai = interactive.AzimuthalIntegrator(
    dist=experimental_setup["ccd_dist"],
    detector=detector,
    wavelength=experimental_setup["lambda"],
    poni1=center[0]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # y (vertical)
    poni2=center[1]
    * experimental_setup["px_size"]
    * experimental_setup["binning"],  # x (horizontal)
)

In [ ]:
# Do 2d Azimuthal integration of all images and add to xarray
list_i2d = []
for im in tqdm(data["images"].values):
    i2d, q, chi = ai.integrate2d(im, 500, 90, dummy=np.nan, mask=mask)
    list_i2d.append(i2d)

# Setup xarray
data["q"] = q
data["chi"] = chi
data["i2d"] = xr.DataArray(list_i2d, dims=["index", "chi", "q"])
data = data.assign_attrs({"center": center})